# Sentiment Analysis Demo

## Description

This notebook provides an interactive demo for performing sentiment analysis using pre-trained Llama models from Hugging Face's `transformers` library, integrated with `gradio` for a user-friendly interface.

### Main Features:
- Select from a variety of pre-trained Llama models for sentiment analysis.
- Input any text and analyze its sentiment (e.g., positive, negative, neutral).
- Display the sentiment result along with the confidence score.
- Cache models to avoid reloading.

## Log in to Hugging Face Hub

In this cell, we import the `login` function from the Hugging Face Hub and call it to authenticate with your Hugging Face account. This step is required to access private models, datasets, and other resources hosted on Hugging Face.

### Directions to Generate Access Token:
1. Go to the [Hugging Face website](https://huggingface.co/).
2. Log in to your Hugging Face account.
3. Navigate to your **profile icon** on the top right, and click **Settings**.
4. Under **Access Tokens** (on the left sidebar), click **New Token** to generate a new access token.
5. Copy the generated token.

### Usage:
When you run this cell, you'll be prompted to paste the access token, which grants access to your Hugging Face resources.

> **Do not share your Access Tokens with anyone**

In [ ]:
from huggingface_hub import login
login()

## Import Required Libraries

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [ ]:
device = 0 if torch.cuda.is_available() else -1

## Basic Usage

Below is a list of available Llama models for sentiment analysis. The function `load_sentiment_model` loads the selected model and its tokenizer using Hugging Face's `AutoModelForSequenceClassification` and `AutoTokenizer`. It returns a sentiment analysis pipeline that can be used to analyze the sentiment of the provided text. Note that, for each version of Llama you need to seperately request access to be able to use them.

In [ ]:
llama_models = {
    "Llama 3 70B Instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "Llama 3 8B Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "Llama 3.1 70B Instruct": "meta-llama/Llama-3.1-70B-Instruct",
    "Llama 3.1 8B Instruct": "meta-llama/Llama-3.1-8B-Instruct",
    "Llama 3.2 3B Instruct": "meta-llama/Llama-3.2-3B-Instruct",
    "Llama 3.2 1B Instruct": "meta-llama/Llama-3.2-1B-Instruct",
}

In [ ]:
def load_sentiment_model(model_name):
    """Load the specified Llama sentiment model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    sentiment_analyzer = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)
    return sentiment_analyzer

Cache models to avoid reloading

In [ ]:
model_cache = {}

The `analyze_sentiment` function performs sentiment analysis using the selected Llama model. It first checks if the model is cached and loads it if not. The function then uses the cached model to analyze the input text, returning the sentiment label (e.g., "positive" or "negative") along with the confidence score.

In [ ]:
def analyze_sentiment(text, model_choice):
    """Perform sentiment analysis using the selected Llama model."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_sentiment_model(llama_models[model_choice])
    sentiment_analyzer = model_cache[model_choice]
    result = sentiment_analyzer(text)[0]
    return f"Sentiment: {result['label']} (Confidence: {result['score']:.2f})"

The cell below defines a simple Gradio interface. The interface consists of a dropdown menu where users can select a specific Llama model for sentiment analysis, a textbox where users can input text for sentiment evaluation, and an output box to display the sentiment result, including the label (e.g., positive or negative) and the confidence score.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Sentiment Analysis with Llama Models</center></h1>")
    model_choice = gr.Dropdown(list(llama_models.keys()), label="Select Llama Model")
    input_text = gr.Textbox(label="Enter text for sentiment analysis", lines=4)
    output_text = gr.Textbox(label="Sentiment Result")
    gr.Button("Analyze").click(analyze_sentiment, [input_text, model_choice], output_text)

Launch the interface

In [ ]:
demo.launch()